<a href="https://colab.research.google.com/github/achmadzano/Machine-Learning/blob/main/Machine_Learning_using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=5415485f958563234c72d190743d0fdb48ebf39add920d0399cb4a979ced92ab
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .getOrCreate()
spark

In [3]:
import pyspark.sql  as pyspark_sql
from pyspark.sql.functions import when
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [6]:
# spark

spark = pyspark_sql.SparkSession.builder.getOrCreate()

# load data
df_train = spark.read.option('inferSchema', 'true').csv('PlaneDelayTraining.csv', header=True)
df_test = spark.read.option('inferSchema', 'true').csv('PlaneDelayTesting.csv', header=True)

# preprocessing
df_train = df_train.na.drop()
df_test = df_test.na.drop()

# select feature
df_train = df_train.select('TimeDeparture', 'LengthOfFlight', 'Airline', 'IsDelayed')
df_test = df_test.select('TimeDeparture', 'LengthOfFlight', 'Airline', 'IsDelayed')

# transform

def transform(df):
    df = df.withColumn('Airline', when(df['Airline']=='DL', 0).when(df['Airline']=='FL',1).when(df['Airline']=='EV', 2))
    return df

df_train = transform(df_train)
df_test = transform(df_test)

# normalize
cols = df_train.columns
cols.remove('IsDelayed')

assembler = VectorAssembler(inputCols=cols, outputCol='Features')
scaler = StandardScaler(inputCol='Features', outputCol='ScaledFeatures')

df_train = assembler.transform(df_train)
df_train = scaler.fit(df_train).transform(df_train)

df_test = assembler.transform(df_test)
df_test = scaler.fit(df_test).transform(df_test)

# generate model
model = LogisticRegression(featuresCol='ScaledFeatures', labelCol='IsDelayed').fit(df_train)
prediction = model.transform(df_test)

# evaluate
evaluator = BinaryClassificationEvaluator(labelCol='IsDelayed')
accuracy = round(evaluator.evaluate(prediction)*100, 2)
print(accuracy, '%')

81.47 %
